In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize

In [6]:
df = pd.read_csv('email1.csv')
df

,Unnamed: 0,file,message
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...,...
9995,9995,bass-e/_sent_mail/1047.,Message-ID: <21376194.1075854731510.JavaMail.e...
9996,9996,bass-e/_sent_mail/1048.,Message-ID: <2401643.1075854731531.JavaMail.ev...
9997,9997,bass-e/_sent_mail/1049.,Message-ID: <32002412.1075854731553.JavaMail.e...
9998,9998,bass-e/_sent_mail/105.,Message-ID: <21429983.1075854709202.JavaMail.e...


In [3]:
pf = pd.DataFrame(df)
pf.drop(pf.index[10000:517401], inplace=True)
pf


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
9995,bass-e/_sent_mail/1047.,Message-ID: <21376194.1075854731510.JavaMail.e...
9996,bass-e/_sent_mail/1048.,Message-ID: <2401643.1075854731531.JavaMail.ev...
9997,bass-e/_sent_mail/1049.,Message-ID: <32002412.1075854731553.JavaMail.e...
9998,bass-e/_sent_mail/105.,Message-ID: <21429983.1075854709202.JavaMail.e...


In [5]:
pf.to_csv('email1.csv')

In [7]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    # keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        # else:
        #     pairs = line.split(':')
        #     key = pairs[0].lower()
        #     val = pairs[1].strip()
        #     # if key in keys_to_extract:
        #     #     email[key] = val
    return email
    
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results    
    
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'), 
        # 'to': map_to_list(emails, 'to'), 
        # 'from_': map_to_list(emails, 'from')
    }

In [8]:
email_df = pd.DataFrame(parse_into_emails(df.message))
email_df

,body
0,Here is our forecast
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,Can you send me a schedule of the salary..."
4,
...,...
9995,"shes pretty sexy, huh? are we getting togethe..."
9996,i copied your idea - and it screwed up your name!
9997,"---------------------------Eric,Just a reminde..."
9998,did you buy any enron in the 60s?


In [6]:
email_df.drop(email_df.query(
    "body == ''"
).index, inplace=True)

In [7]:
email_df. reset_index(inplace=True)

PRE-PROCESSING

In [8]:
import re #library for simplifying reviews
nltk.download('stopwords') #downloading all the stopwords
from nltk.corpus import stopwords #importing stopwords to our notebook
from nltk.stem.porter import PorterStemmer #Stemming is a process that converts a word to its root eg loved => love, stayed => stay.
# this is necessary to optimise the dimention of the sparse matrix that will be made.
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DiyaK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
corpus = [] # new list that will contain all the cleaned reviews
corpus1 = []
for i in range(len(email_df)):
    #remove punctuation by replacing all punctuation with space
    review = re.sub("[^a-zA-z | ^\w+'t]",' ', email_df['body'][i])
    
    #transform all capital letters to lower case
    review = review.lower()
    
    #split review to different words to apply stemming
    review = review.split()
    
    #stemming and also ignoring stopwords alongside
    ps = PorterStemmer()
    wordnet = WordNetLemmatizer()
    all_stopwords=stopwords.words('english')
    r_words=['no','not','off','nor',"didn't","isn't","couldn't","haven't",'or',"should've","aren't",
             "couldn","didn","doesn't",'doesn',"don't",'don','hadn',"hadn't",'hasn',"hasn't",'haven',
             'mightn',"mightn't",'mustn',"mustn't","needn't",'needn',"shouldn","shouldn't",'wasn',
             "wasn't","won't","won","wouldn't","because","same",'wouldn','should']
    review = [wordnet.lemmatize(word) for word in review if not word in set(all_stopwords)]
    #joining words together to form a cleaned review
    for i in review:
        corpus.append(i)
    review = ' '.join(review)
    corpus1.append(review)
   

In [10]:

len(corpus)


1369325

In [11]:
from nltk.probability import FreqDist
fdist = FreqDist()

In [12]:
for word in corpus1:
    fdist[word] += 1
fdist    


FreqDist({'george huan enron com mike maggi enron com larry may enron com hal mckinney enron com errol mclaughlin enron com dutch quigley enron com sean riordan enron com joey taylor enron com dan thibaut enron com': 46, 'information contained herein based source believe bereliable represent accurate complete nothingcontained herein considered offer sell solicitationof offer buy financial instrument discussed herein anyopinions expressed herein solely author theymay differ material respect expressed published byon behalf carr future officer director employee oraffiliates 2000 carr futuresthe chart available web clicking hot link contained email reason unable receive thecharts via web please contact via email email chartsto attachment nat gas strip matrixnat gas spread matrixcrude product spread matrixcarr futures150 wacker dr suite 1500chicago il 60606 usasoblander carrfut com': 20, 'following report waiting approval 4 day please review': 19, 'information contained herein based source 

In [13]:
for word in corpus:
    fdist[word] += 1
fdist 

FreqDist({'enron': 39824, 'com': 28608, 'ect': 13745, 'hou': 8297, '1': 8080, 'td': 5885, 'font': 5586, 'company': 4949, 'e': 4928, 'would': 4782, ...})

In [14]:
fdist.most_common(1000)

[('enron', 39824),
 ('com', 28608),
 ('ect', 13745),
 ('hou', 8297),
 ('1', 8080),
 ('td', 5885),
 ('font', 5586),
 ('company', 4949),
 ('e', 4928),
 ('would', 4782),
 ('message', 4569),
 ('please', 4555),
 ('market', 4401),
 ('said', 4395),
 ('2', 4307),
 ('09', 4248),
 ('gas', 4141),
 ('corp', 3949),
 ('price', 3813),
 ('u', 3594),
 ('time', 3543),
 ('right', 3533),
 ('power', 3472),
 ('year', 3358),
 ('new', 3314),
 ('mail', 3218),
 ('original', 3169),
 ('energy', 3164),
 ('get', 3139),
 ('john', 3053),
 ('size', 3030),
 ('3', 2916),
 ('ee', 2873),
 ('20', 2808),
 ('one', 2787),
 ('align', 2778),
 ('5', 2670),
 ('may', 2647),
 ('phillip', 2630),
 ('week', 2564),
 ('day', 2542),
 ('0', 2539),
 ('need', 2508),
 ('b', 2508),
 ('call', 2468),
 ('10', 2460),
 ('know', 2457),
 ('2001', 2377),
 ('also', 2325),
 ('4', 2261),
 ('like', 2240),
 ('service', 2225),
 ('business', 2207),
 ('3d', 2193),
 ('[image]', 2140),
 ('email', 2138),
 ('tr', 2124),
 ('information', 2096),
 ('p', 2052),
 ('2

In [15]:
from collections import Counter
Counter(corpus)

Counter({'forecast': 285,
         'traveling': 43,
         'business': 2207,
         'meeting': 1650,
         'take': 1387,
         'fun': 180,
         'trip': 160,
         'especially': 189,
         'prepare': 91,
         'presentation': 254,
         'would': 4782,
         'suggest': 99,
         'holding': 142,
         'plan': 1329,
         'without': 463,
         'formal': 137,
         'even': 793,
         'try': 488,
         'get': 3139,
         'honest': 26,
         'opinion': 161,
         'whether': 439,
         'desired': 29,
         'necessary': 270,
         'far': 412,
         'think': 1753,
         'productive': 13,
         'stimulate': 7,
         'discussion': 419,
         'across': 230,
         'different': 421,
         'group': 1256,
         'working': 571,
         'often': 143,
         'presenter': 8,
         'speaks': 14,
         'others': 266,
         'quiet': 19,
         'waiting': 139,
         'turn': 228,
         'might': 758,
 

MemoryError: Unable to allocate 679. GiB for an array with shape (1369325, 66582) and data type int64